In [24]:
from text2graph.modules import (
    OllamaClient,
    OllamaMessage,
    OllamaResponse,
    OllamaGenerateResponse,
    OllamaError,
    chat,
)
import nltk

In [ ]:
client = OllamaClient(host="http://localhost:11434")
LLM_model = "granite4:small-h"
#response = client.chat(messages=[system_msg.to_json(), assistant_msg.to_json(), user_msg.to_json()], model="llama3", stream=False)
#print("Content:", response.content)
#print("Parsed data:", response.parsed_data)

In [43]:
result_list = []
the_text = "Standard Retrieval-Augmented Generation (RAG) relies on chunk-based retrieval, whereas GraphRAG advances this approach by graph-based knowledge representation. However, existing graph-based RAG approaches are constrained by binary relations, as each edge in an ordinary graph connects only two entities, limiting their ability to represent the n-ary relations (n >= 2) in real-world knowledge. In this work, we propose HyperGraphRAG, a novel hypergraph-based RAG method that represents n-ary relational facts via hyperedges, and consists of knowledge hypergraph construction, retrieval, and generation. Experiments across medicine, agriculture, computer science, and law demonstrate that HyperGraphRAG outperforms both standard RAG and previous graph-based RAG methods in answer accuracy, retrieval efficiency, and generation quality. Our data and code are publicly available at this https URL."

In [140]:
class topicMap:
    locations: list[int] = []
    text: str = ""
    def __init__(self, text: str, new_locations: list[int]):
        self.text = text
        self.locations = new_locations
    
    def __str__(self) -> str:
        return f"Topic: {self.text}, Locations: {self.locations}"
    
    def __repr__(self) -> str:
        return self.__str__()
    
    def __eq__(self, text: object) -> bool:
        return self.text == text

In [141]:
LLM_model = "glm-4.7-flash"
LLM_model = "glm-4.7-flash:q8_0"
LLM_model = "qwen3.5:397b-cloud"
#LLM_model = "glm-5:cloud"
system_msg = {"role": "system", "content": """You are a helpful assistant that extracts topics and relations from a text to create a knowledge graph.

INSTRUCTIONS:
1. Extract all key topics/entities mentioned in the text
2. Identify meaningful relationships between these topics
3. Use the exact topic labels as they appear in the text for sourceTopic and targetTopic fields
4. Choose appropriate relation types that describe how the topics are connected
5. Provide categories for topics when possible (method, technology, concept, field, etc.)
6. Include brief descriptions for complex topics
7. Return ONLY valid JSON that matches the provided schema structure

SCHEMA REQUIREMENTS:
- topics: Array of unique topics with label, category, and description
- edges: Array of relationships with sourceTopic, targetTopic, relation, and optional weight
- All topic labels referenced in edges must exist in the topics array
- Use descriptive relation names like 'uses', 'improves', 'compares', 'outperforms', 'represents'"""}
user_prompt = f"""
Analyze the following text and extract a knowledge graph with topics and their relationships.

STEPS:
1. Identify all key topics/entities in the text
2. Determine relationships between these topics
3. Create a JSON structure with:
   - topics: list of unique topics with labels, categories, and descriptions
   - edges: list of relationships between topics

TEXT TO ANALYZE:
{the_text}

Return only the JSON structure matching the provided schema."""

In [112]:
def locator(text: str, topic: str) -> topicMap:
    locations = []
    start = 0
    while True:
        location = text.find(topic, start)
        if location == -1:
            break
        locations.append(location)
        start = location + len(topic)

    ting = topicMap(topic, locations)
    return ting

In [113]:
x = locator(the_text, "Standard Retrieval-Augmented Generation (RAG)")
#print(x)
x 

Topic: Standard Retrieval-Augmented Generation (RAG), Locations: [0]

In [131]:
print(EdgeList.model_json_schema())

{'$defs': {'Edge': {'properties': {'Source_Topic': {'title': 'Source Topic', 'type': 'string'}, 'Target_Topic': {'title': 'Target Topic', 'type': 'string'}, 'Relation': {'title': 'Relation', 'type': 'string'}, 'Weight': {'anyOf': [{'type': 'number'}, {'type': 'null'}], 'title': 'Weight'}}, 'required': ['Source_Topic', 'Target_Topic', 'Relation', 'Weight'], 'title': 'Edge', 'type': 'object'}}, 'properties': {'edges': {'items': {'$ref': '#/$defs/Edge'}, 'title': 'Edges', 'type': 'array'}}, 'required': ['edges'], 'title': 'EdgeList', 'type': 'object'}


In [117]:
topic_indexs = []
for edge in edge_list.edges:
    print(f"{edge.Source_Topic} --{edge.Relation}--> {edge.Target_Topic} (weight: {edge.Weight})")
    #print(f"Source_Topic '{edge.Source_Topic}' found at index: {location}")
    if edge.Source_Topic not in topic_indexs:
        topic_indexs.append(locator(the_text, edge.Source_Topic))
    if edge.Target_Topic not in topic_indexs:
        topic_indexs.append(locator(the_text, edge.Target_Topic))
    #print('-'*30)
    #break

topic_indexs


Standard Retrieval-Augmented Generation (RAG) --relies on--> chunk-based retrieval (weight: 1.0)
GraphRAG --advances--> Standard Retrieval-Augmented Generation (RAG) (weight: 1.0)
GraphRAG --uses--> graph-based knowledge representation (weight: 1.0)
existing graph-based RAG approaches --are constrained by--> binary relations (weight: 1.0)
ordinary graph --edges connect only two--> entities (weight: 1.0)
binary relations --limit ability to represent--> n-ary relations (weight: 1.0)
HyperGraphRAG --represents via hyperedges--> n-ary relational facts (weight: 1.0)
HyperGraphRAG --consists of--> knowledge hypergraph construction (weight: 1.0)
HyperGraphRAG --consists of--> retrieval (weight: 1.0)
HyperGraphRAG --consists of--> generation (weight: 1.0)
Experiments --demonstrate outperforms standard RAG--> HyperGraphRAG (weight: 1.0)
Experiments --demonstrate outperforms previous graph-based RAG methods--> HyperGraphRAG (weight: 1.0)
HyperGraphRAG --outperforms in answer accuracy, retrieval 

Standard Retrieval-Augmented Generation (RAG) --relies on--> chunk-based retrieval (weight: 1.0)
GraphRAG --advances--> Standard Retrieval-Augmented Generation (RAG) (weight: 1.0)
GraphRAG --uses--> graph-based knowledge representation (weight: 1.0)
existing graph-based RAG approaches --are constrained by--> binary relations (weight: 1.0)
ordinary graph --edges connect only two--> entities (weight: 1.0)
binary relations --limit ability to represent--> n-ary relations (weight: 1.0)
HyperGraphRAG --represents via hyperedges--> n-ary relational facts (weight: 1.0)
HyperGraphRAG --consists of--> knowledge hypergraph construction (weight: 1.0)
HyperGraphRAG --consists of--> retrieval (weight: 1.0)
HyperGraphRAG --consists of--> generation (weight: 1.0)
Experiments --demonstrate outperforms standard RAG--> HyperGraphRAG (weight: 1.0)
Experiments --demonstrate outperforms previous graph-based RAG methods--> HyperGraphRAG (weight: 1.0)
HyperGraphRAG --outperforms in answer accuracy, retrieval 

[Topic: Standard Retrieval-Augmented Generation (RAG), Locations: [0],
 Topic: chunk-based retrieval, Locations: [56],
 Topic: GraphRAG, Locations: [87, 424, 693],
 Topic: graph-based knowledge representation, Locations: [122],
 Topic: existing graph-based RAG approaches, Locations: [169],
 Topic: binary relations, Locations: [224],
 Topic: ordinary graph, Locations: [261],
 Topic: entities, Locations: [294],
 Topic: n-ary relations, Locations: [344],
 Topic: HyperGraphRAG, Locations: [419, 688],
 Topic: n-ary relational facts, Locations: [486],
 Topic: knowledge hypergraph construction, Locations: [541],
 Topic: retrieval, Locations: [68, 576, 789],
 Topic: generation, Locations: [591, 815],
 Topic: Experiments, Locations: [603],
 Topic: standard RAG, Locations: [719],
 Topic: previous graph-based RAG methods, Locations: [736],
 Topic: data and code, Locations: [839],
 Topic: URL, Locations: [890]]